In [1]:
import numpy as np
import pandas as pd
import mysql.connector
import re

In [2]:
try:
    with open('user_pass.txt', 'r') as fp:
        lines = fp.readlines()

    user = lines[0].split('\"')[-2]
    password = lines[1].split('\"')[-2]
except:
    print("""The User Pass format should be like:
            user = "myUser"
            pass =  "myPassword"  
            """)

In [3]:
db = mysql.connector.connect(
    host = "localhost",
    user = f"{user}",
    password = f"{password}",
)

In [4]:
mycursor = db.cursor()

In [5]:
SQL_queries = ["DROP DATABASE IF EXISTS `IMDB_Movies`;"
        , "CREATE DATABASE `IMDB_Movies`;"
        , "USE `IMDB_Movies`;"
        , """CREATE TABLE Top250_Movies(
        movie_id int(4) NOT NULL AUTO_INCREMENT
        , Movie_Name varchar(100) NOT NULL
        , Year int(4) NOT NULL
        , IMDB_Rating float(4) NOT NULL
        , PRIMARY KEY (movie_id)
        ) ENGINE=InnoDB AUTO_INCREMENT=11 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;"""
        , "ALTER TABLE Top250_Movies AUTO_INCREMENT = 1;"]

for query in SQL_queries:
    mycursor.execute(query)
    

In [6]:
def load_csv(addr):
    return pd.read_csv(addr)

In [7]:
addr = r".\Top250_Movies_Info_cleaned.csv"
top250_df = load_csv(addr)
top250_df.tail(3)

,Title,Year,Certificate,Time,Genere,plot,Rating,Voting,Stars,Directors,Writers,User reviews,Critic reviews,Metascore,URL,minutes
247,The Passion of Joan of Arc,1928,A,1h 50min,"['Biography', 'Drama', 'History']","In 1431, Jeanne d'Arc is placed on trial on ch...",8.1,50000.0,"['Maria Falconetti', 'Eugene Silvain', 'André ...",['Carl Theodor Dreyer'],"['Joseph Delteil', 'Carl Theodor Dreyer']",210,141,82.660465,https://www.imdb.com//title/tt0019254/,110.0
248,Sunrise: A Song of Two Humans,1927,A,1h 34min,"['Drama', 'Romance']",A sophisticated urban woman seduces a farmer i...,8.1,49000.0,"[""George O'Brien"", 'Janet Gaynor', 'Margaret L...",['F.W. Murnau'],"['Carl Mayer', 'Hermann Sudermann', 'Katherine...",270,122,82.660465,https://www.imdb.com//title/tt0018455/,94.0
249,The General,1926,U,1h 7min,"['Action', 'Adventure', 'Comedy']",When Union spies steal an engineer's beloved l...,8.1,85000.0,"['Buster Keaton', 'Marion Mack', 'Glen Cavende...","['Clyde Bruckman', 'Buster Keaton']","['Buster Keaton', 'Clyde Bruckman', 'Al Boasbe...",311,120,82.660465,https://www.imdb.com//title/tt0017925/,67.0


In [8]:
for ind, movie_Info in top250_df.iterrows():
    #print(movie_Info)
    title = movie_Info['Title']
    year = int(movie_Info['Year'])
    rating = movie_Info['Rating']
    title = title.replace('\'', '')
    query = f"INSERT INTO Top250_Movies (Movie_Name, Year, IMDB_Rating) VALUES ('{title}', '{year}', '{rating}')"
    #print(query)
    mycursor.execute(query)

# Make sure data is committed to the database
db.commit()

In [19]:
mycursor.execute("""SELECT *
FROM Top250_movies""")

y = []
for ind, x in enumerate(mycursor): 
    y.append(x)

table_df = pd.DataFrame(y, columns =mycursor.column_names)
table_df.head(10)

,movie_id,Movie_Name,Year,IMDB_Rating
0,1,The Shawshank Redemption,1994,9.3
1,2,The Godfather,1972,9.2
2,3,The Dark Knight,2008,9.0
3,4,The Godfather: Part II,1974,9.0
4,5,12 Angry Men,1957,9.0
5,6,The Lord of the Rings: The Return of the King,2003,8.9
6,7,Pulp Fiction,1994,8.9
7,8,Schindlers List,1993,8.9
8,9,Inception,2010,8.8
9,10,Fight Club,1999,8.8
